## AutoGen Swarm
Swarm implements a team in which agents can hand off task to other agents based on their capabilities. It is a multi-agent design pattern first introduced by OpenAI in an experimental project. The key idea is to let agent delegate tasks to other agents using a special tool call, while all agents share the same message context. This enables agents to make local decisions about task planning, rather than relying on a central orchestrator such as in SelectorGroupChat.

The speaker agent is selected based on the most recent HandoffMessage message in the context. This naturally requires each agent in the team to be able to generate HandoffMessage to signal which other agents that it hands off to.

In general, this is how it works:

1. Each agent has the ability to generate HandoffMessage to signal which other agents it can hand off to. For AssistantAgent, this means setting the handoffs argument.

2. When the team starts on a task, the first speaker agents operate on the task and make locallized decision about whether to hand off and to whom.

3. When an agent generates a HandoffMessage, the receiving agent takes over the task with the same message context.

4. The process continues until a termination condition is met.

## Load Azure Configurations

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

azure_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## Creating the Agents

In [ ]:
import arxiv
import json
import os

from langchain_community.document_loaders import PyPDFLoader
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

PAPER_DIR = "../ArxivResearcher/swarm_papers"

# Create the user proxy agent.
user_proxy_agent = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Use arxiv to find the papers 
def search_arxiv(topic: str, max_results: int = 5):
    client = arxiv.Client()

    # Search for the latest articles matching the queried topic
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    papers = client.results(search)
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
            return file_path  # If file exists, return the path without updating
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    paper_ids = []
    for paper in papers:
        paper_id = paper.entry_id.split('/')[-1] # Use the last part of the entry_id as a unique identifier
        paper_ids.append(paper_id)
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper_id] = paper_info

    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json_file.write(json.dumps(papers_info, indent=2))
    print(f"Papers metadata saved to {file_path}")

    return file_path

# Create the arxiv search agent.
arxiv_search_agent = AssistantAgent(
    name="arxiv_search",
    description="An agent that can retrieve research papers related to user input on arXiv \
        and save the metadata of the papers in a JSON file.",
    handoffs=["content_extractor", "planning_agent"],
    model_client=azure_model_client,
    tools=[search_arxiv],
    system_message="You are an expert academic search assistant. \
        Your task is to find the most relevant and recent papers on arXiv \
        for given user input, henceforth called topic, and stores them in JSON file 'papers_info.json'.\
        Once the task is complete, handoff to the content_extractor_agent."
)
# Create the content extractor agent that downloads the PDFs and extracts the content.
def content_extractor(file_path: str):

    with open(file_path, "r") as json_file:
        papers_info = json.load(json_file)

    papers_contents = {}
    # Iterate through each paper's info and download the PDF
    for paper_id, info in papers_info.items():
        pdf_path = os.path.join(os.path.dirname(file_path), f"{paper_id}.pdf")
        
        # Download the PDF if it doesn't exist
        if not os.path.exists(pdf_path):
            paper = next(arxiv.Client().results(arxiv.Search(id_list=[paper_id])))
            paper.download_pdf(filename=pdf_path)

        # Load the PDF and extract text
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()

        # Extract the raw text from the document
        if documents:
            papers_contents[paper_id] = documents[0].page_content[:5000]  # Limit to first 5000 characters for brevity
        else:
            papers_contents[paper_id] = "No content found."

    # Save the extracted contents to a JSON file
    contents_file_path = os.path.join(os.path.dirname(file_path), "papers_contents.json")
    with open(contents_file_path, "w") as contents_file:
        json.dump(papers_contents, contents_file, indent=2)

    return contents_file_path

# Create the Content Extractor agent.
content_extractor_agent = AssistantAgent(
    name="content_extractor",
    description="An agent that extracts raw text from all the downloaded arxiv research papers \
    obtained from the arxiv search agent.",
    handoffs=["research_summarizer","planning_agent"],
    model_client=azure_model_client,
    tools=[content_extractor],
    system_message="You are a research papers content extractor. \
        Read paper_id from papers_info.json. Download PDFs and extract the raw text content from each paper. \
        Ignore the papers whose content could not be extracted.\
        The extracted contents should be saved in a JSON file named 'papers_contents.json'.\
        Return the path to the JSON file and handoff to the research_summarizer_agent."
)

# Function to generate full text from the extracted papers contents.
def generate_full_text(content_file_path: str):
    """Combine the raw texts of the papers from the JSON file."""
    with open(content_file_path, "r") as json_file:
        papers_contents = json.load(json_file)
    full_text = "\n".join(papers_contents.values())
    print(f"Read {len(papers_contents)} papers contents from {content_file_path}.")
    print(f"Total characters in papers contents: {len(full_text)}")
    
    return full_text

# Create the Research Summarizer agent.
research_summarizer_agent = AssistantAgent(
    name="research_summarizer",
    description="An agent that writes a single coherent summary on a topic from the top 5 research papers on arXiv.",
    handoffs=["summary_writer","planning_agent"],
    model_client=azure_model_client,
    tools=[generate_full_text], 
    system_message="You are an expert academic researcher. \
        Your task is to write a single coherent summary relevant to the given topic in under 5000 words. \
        To do this, first generate full text from the contents of all papers by reading \
        the JSON file 'papers_contents.json' located in the topic's subdirectory inside the project directory. \
        The file contains the paper_ids and raw text content from each paper. \
        Focus on common themes and significant advancements. \
        Do not just summarize the papers individually, Always address the topic at hand.\
        Once the summary is ready, handoff to the summary_writer_agent."   
)

# Create the summary writer agent that writes the generated summary to a text file.
def write_summary_to_file(summary: str, topic: str):
    """Write the generated summary to a text file named 'summary.txt' in the topic's subdirectory."""
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    summary_file_path = os.path.join(path, "summary.txt")
    
    with open(summary_file_path, "w") as summary_file:
        summary_file.write(summary)
    
    print(f"Summary written to {summary_file_path}")
    return summary_file_path

summary_writer_agent = AssistantAgent(
    name="summary_writer",
    description="An agent that writes the generated summary to a text file.",
    handoffs=["planning_agent", "research_summarizer"],
    model_client=azure_model_client,
    tools=[write_summary_to_file],
    system_message="You are a summary writer. \
        Your task is to write the given summary to a text file named 'summary.txt' in the topic's subdirectory.\
        if the summary is not ready, handoff to the research_summarizer_agent. \
        If the summary is ready, write it to the given text file path and handoff to the planning_agent."
)

In [ ]:
# create the planning agent
planning_agent = AssistantAgent(
    "planning_agent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    handoffs=["user_proxy", "arxiv_search", "content_extractor", "research_summarizer", "summary_writer"],
    model_client=azure_model_client,
    system_message=f"""
    You are a planning agent that coordinates the work of your team members.
    Coordinate handoff to specialized agents:
    - user_proxy: for receiving user input and relaying it to the team.
    - arxiv_search-agent: for searching arxiv for relevant research papers based on user input.
    - info_extractor_agent: for extracting raw text from the downloaded research papers.
    - research_summarizer_agent: for generating a coherent summary based on the contents of the papers.
    - summary_writer_agent: for writing the generated summary to a text file.

    You will follow this sequence:
        You will prompt the user for a topic of interest.
        arxiv_search_agent will retrieve top 5 research papers based on the initial user prompt and save their metadata in papers_info.json.
        info_extractor_agent will extract the raw text from the downloaded papers and save it in papers_contents.json.
        research_summarizer_agent will generate a coherent summary based on the contents of the papers.
        summary_writer_agent will write the generated summary to a text file.
     
    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time.
    """,
)

## Creating the Team

In [5]:
text_mention_termination = TextMentionTermination("QUIT")
max_messages_termination = MaxMessageTermination(max_messages=50)
termination = text_mention_termination | max_messages_termination

team = Swarm(
    participants=[planning_agent,  user_proxy_agent, arxiv_search_agent, content_extractor_agent, \
     research_summarizer_agent, summary_writer_agent], 
    termination_condition=termination
)

## Specify the Task and Run the Team

In [ ]:
# Stream the messages to the console.
await Console(team.run_stream())  
#await team.reset()  # Reset the team for the next run.

---------- ThoughtEvent (planning_agent) ----------
To begin, I will coordinate the retrieval of relevant research papers based on the user's input. Here’s the plan:

1. Handoff to the **arxiv_search_agent** to search for the top 5 research papers that are relevant to the user's prompt. The metadata of these papers will be saved in `papers_info.json`.
2. Once the search is complete, I will hand off to the **info_extractor_agent** to extract the raw text from the downloaded research papers, saving it in `papers_contents.json`.
3. After the content extraction, I will coordinate with the **research_summarizer_agent** to generate a coherent summary based on the extracted contents.
4. Finally, I will pass the generated summary to the **summary_writer_agent** to save it in a text file.

Now, I will proceed by handing off to the **arxiv_search_agent**. 

Let's start! 

Handoff to arxiv_search_agent.
---------- ToolCallRequestEvent (planning_agent) ----------
[FunctionCall(id='call_f8bVS4dvtKf